In [22]:
import pandas as pd
from matricesRalas import *

In [23]:
papers = pd.read_csv('papers/papers.csv', header = 0)
citas = pd.read_csv('papers/citas.csv', header = 0)

In [24]:
W = MatrizRala(len(papers), len(papers))
D = MatrizRala(len(papers), len(papers))

In [25]:
cantidad_citas = [0] * len(papers)
for index, row in citas.iterrows():
    W[row['to'], row['from']] = 1
    cantidad_citas[row['from']] += 1

for i in range(len(cantidad_citas)):
    if cantidad_citas[i] != 0:
        D[i, i] = 1/cantidad_citas[i]

In [26]:
def matriz_de_unos(n,m):
    matriz = MatrizRala(n,m)
    for i in range(matriz.shape[0]):
        for j in range(matriz.shape[1]):
            matriz[i,j] = 1
    return matriz

In [27]:
def P_it(d,N,W,D):
    p_t = MatrizRala(N,1)     # Initial equiprobable distribution
    for i in range(N):
        p_t[i,0] = 1/N

    tolerance = 1e-6
    errores = []
    error = 1

    mat_unos = matriz_de_unos(N,1)
    unoMenosDeSobreEne = ((1-d)/N) * mat_unos
    d_W = d * W
    d_WD = d_W @ D


    while error > tolerance:
        # Multiplica la matriz W_D por el vector p_t y escala por d
        p_t_plus_1 = d_WD @ p_t
        p_t_plus_1 = unoMenosDeSobreEne + p_t_plus_1
        # Calcula el error máximo en esta iteración comparando el nuevo vector de PageRank con el anterior
        error = max(abs(p_t_plus_1[i,0] - p_t[i,0]) for i in range(N))
        errores.append(error)

        # Actualiza el vector de PageRank para la próxima iteración
        p_t = p_t_plus_1
    return p_t, errores

In [28]:

# Llamar a la función y pasar la ruta al archivo CSV

N = len(papers)
d = 0.85

page_ranks = P_it(d, N, W, D)

# Create list of (PageRank score, index)
lista = [(page_ranks[0][i, 0], i) for i in range(len(papers))]

# Sort by PageRank score in descending order
sorted_papers = sorted(lista, key=lambda x: x[0], reverse=True)

# Print the top 10 papers with podium ranking
print("Top 10 Papers by PageRank:")
for rank, (score, paper_id) in enumerate(sorted_papers[:10], start=1):
    paper = papers.loc[paper_id]
    print(f"{rank}. Paper ID: {paper['id']}, Title: \"{paper['titulo']}\", Score: {score}")

Top 10 Papers by PageRank:
1. Paper ID: 81323, Title: "The art of computer programming, volume 2 (3rd ed.): seminumerical algorithms", Score: 0.00010947382752488224
2. Paper ID: 327827, Title: "A method for obtaining digital signatures and public-key cryptosystems", Score: 0.00010919870666871219
3. Paper ID: 79620, Title: "The art of computer programming, volume 1 (3rd ed.): fundamental algorithms", Score: 0.00010681341007934271
4. Paper ID: 326368, Title: "A relational model of data for large shared data banks", Score: 0.00010487915217928264
5. Paper ID: 547167, Title: "Recovery semantics for a DB/DC system", Score: 0.00010371568503380856
6. Paper ID: 552437, Title: "Recovery scenario for a DB/DC system", Score: 0.00010206767474799947
7. Paper ID: 322720, Title: "Programming semantics for multiprogrammed computations", Score: 6.540565780259542e-05
8. Paper ID: 153803, Title: "Principles of interactive computer graphics (2nd ed.)", Score: 6.263595635632845e-05
9. Paper ID: 328020, Titl